<a href="https://www.kaggle.com/code/tensorkelechi/shiryoku-tiny?scriptVersionId=179728100" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import models
from tqdm.auto import tqdm
import os
import cv2
import nltk
from datasets import load_dataset
import requests
import PIL.Image as pillow_image
from nltk import tokenize
from torchvision import transforms
import re
from collections import Counter
import numpy as np
from einops import rearrange
from matplotlib import pyplot as plt
print('Imports complete')

Imports complete


In [3]:
class Config:
    num_epochs = 35
    model_output_path = "shiryoku_icm"
    model_filename = "shiryoku_vision.pth"
    lr = 0.01
    momentum = 0.9
    batch_size = 32
    top_k = 2
    num_experts = 4
    image_size = 200
    train_size = 0.95
    embed_size = 512
    hidden_size = 1024

In [4]:
# For image data
def image_transforms(image_file):
    transformed_image = transforms.Compose(
        [
            transforms.RandomCrop(Config.image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

    return transformed_image

import cv2
import os
from PIL import Image
import requests
from io import BytesIO

def read_img(image_data):
    if isinstance(image_data, str):
        # Assume image_data is a file path
        image = cv2.imread(image_data)
    elif isinstance(image_data, bytes):
        # Assume image_data is binary data
        image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
    elif isinstance(image_data, Image.Image):
        # Assume image_data is a PIL Image object
        image = cv2.cvtColor(np.array(image_data), cv2.COLOR_RGB2BGR)
    else:
        # Assume image_data is a URL
        response = requests.get(image_data)
        image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)

    if image is None:
        raise ValueError("Could not read the image data.")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize the image
    return image


def load_image(url, output_path=None):
    try:
        # Attempt to load image from URL
        url_content = requests.get(url, stream=True).raw
        image = pillow_image.open(url_content)
        return image
        
    except Exception as e:
        
#         print(f"Error loading image: {url}, {e}")
        return None
        

#     print(f"success loading {url}")


# For text data
def tokenize_text(text_input):
    text_tokens = tokenize.word_tokenize(str(text_input).lower())

    return text_tokens


def preprocess_text(text):
    text = re.sub(r"[^a-z0-9\s]", "", text)
    tokens = tokenize_text(text)
    tokens = [t for t in tokens if t not in nltk.corpus.stopwords.words("english")]
    return tokens


def create_vocabulary(text_dataset):
    all_tokens = []
    for text in text_dataset:
        tokens = preprocess_text(text)
        all_tokens.extend(tokens)

    vocab_counter = Counter(all_tokens)
    vocab = [word for word, count in vocab_counter.most_common() if count >= 1]
    vocab = ["<pad>", "[s]", "<end>", "_"] + vocab

    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for idx, word in enumerate(vocab)}

    return word_to_idx, idx_to_word


In [5]:
# def load_moondream_dataset():
#     moondream_dataset = load_dataset("isidentical/moondream2-coyo-5M-captions")
#     md_data = moondream_dataset['train'][:100] # type: ignore

#     image_urls = md_data["url"] # type: ignore
#     descriptions = md_data["moondream2_caption"] # type: ignore
    
#     for url, desc in tqdm(zip(image_urls, descriptions)):
#         image = load_image(url)
#         caption = desc.lower()

#         if image is not None:
#             yield (image, caption)


In [6]:
from multiprocessing import Pool

def load_image_wrapper(url):
    try:
        image = load_image(url)
        return image
    except:
        return None

def get_moondream_dataset():
    moondream_dataset = load_dataset("isidentical/moondream2-coyo-5M-captions")
    md_data = moondream_dataset['train'][:1000]
    image_urls = md_data["url"]
    descriptions = md_data["moondream2_caption"]

    with Pool(processes=7) as pool:  # Adjust number of processes as needed
        images = pool.map(load_image_wrapper, image_urls)
        captions = [desc.lower() for desc in descriptions]

    for image, caption in tqdm(zip(images, captions)):
        if image is not None:
            yield (image, caption)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
# images, captions = fetch_moondream_dataset()

cuda


In [8]:
%%time
images, captions = zip(*get_moondream_dataset())

print("data fetch complete")

Generating train split:   0%|          | 0/5005590 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

data fetch complete
CPU times: user 19.9 s, sys: 12.1 s, total: 32 s
Wall time: 3min 1s


In [9]:
captions_vocab = create_vocabulary(captions)

# print(captions_vocab)

In [10]:
print(f'{len(images)} and {len(captions)}')

939 and 939


In [11]:
caption_vocab = captions_vocab[0]

vocab_size = len(caption_vocab)
idx2word = captions_vocab[1]
len(caption_vocab), type(caption_vocab)

(3512, dict)

In [12]:
unk_token = '_'
unk_idx = caption_vocab.get(unk_token, len(caption_vocab))  # Assign a new index if '<unk>' is not present

def tokenize_caption(caption, vocab):
    caption_tokens = tokenize_text(caption)
    caption_indices = []
    for token in caption_tokens:
        caption_indices.append(vocab.get(token, unk_idx))
    return caption_indices

In [13]:
class ImageCaptionData(Dataset):
    def __init__(self, images, captions, captions_vocab=caption_vocab,transforms=None, device=device):
        super().__init__()
        self.images = images
        self.captions = captions
        self.transform = transforms
        self.device = device
        self.vocab = captions_vocab
        self.max_caption_length = max(len(tokenize_caption(caption, captions_vocab)) for caption in captions)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        try:
            image = read_img(self.images[idx])
        except Exception as e:
            print(f"Error reading image at index {idx}: {e}")
            return None, None, None
        
        if self.transform:
            image = self.transform(image)
            
#         print(f"B4 tensorizing: {image.shape}")
        torch.from_numpy(image)
        image = torch.tensor(image, dtype=torch.float32).to(self.device)
        image = rearrange(image, "h w c -> c h w")
#         print(f"After tensorizing: {image.shape}")
        
        # caption
        vocab = self.vocab
        pad_idx = vocab['<pad>']
        
        caption = []
        caption_tokens = tokenize_text(self.captions[idx])
        caption.append(vocab['[s]'])
        caption_tokens = tokenize_caption(self.captions[idx], vocab)
        caption.extend(caption_tokens)
        caption.append(vocab['<end>'])
        max_length = self.max_caption_length
        caption = caption + [pad_idx] * (max_length - len(caption))
        caption = torch.tensor(caption).to(self.device)
        
        length = len(caption)
        
        return image, caption, length

dataset = ImageCaptionData(
    images=images, captions=captions
)

train_size = 80

val_size = len(dataset) - train_size

In [14]:
dataset[1]   

(tensor([[[0.9176, 0.9137, 0.9059,  ..., 0.9059, 0.9098, 0.9216],
          [0.9020, 0.9176, 0.8824,  ..., 0.9176, 0.9255, 0.9216],
          [0.9176, 0.8902, 0.9059,  ..., 0.9255, 0.9333, 0.9216],
          ...,
          [0.9137, 0.8431, 0.8549,  ..., 0.9176, 0.9137, 0.9098],
          [0.8118, 0.8392, 0.8549,  ..., 0.8941, 0.8824, 0.8863],
          [0.8392, 0.8314, 0.8353,  ..., 0.8980, 0.8980, 0.8980]],
 
         [[0.9373, 0.9333, 0.9255,  ..., 0.7490, 0.7529, 0.7725],
          [0.9216, 0.9373, 0.9020,  ..., 0.7608, 0.7686, 0.7725],
          [0.9373, 0.9098, 0.9255,  ..., 0.7686, 0.7765, 0.7725],
          ...,
          [0.7412, 0.6706, 0.6824,  ..., 0.7294, 0.7294, 0.7373],
          [0.6392, 0.6667, 0.6824,  ..., 0.7216, 0.7137, 0.7176],
          [0.6667, 0.6588, 0.6627,  ..., 0.7451, 0.7412, 0.7294]],
 
         [[0.9608, 0.9569, 0.9490,  ..., 0.5490, 0.5529, 0.5804],
          [0.9451, 0.9608, 0.9255,  ..., 0.5608, 0.5686, 0.5804],
          [0.9608, 0.9333, 0.9490,  ...,

In [15]:
def pad_collate(batch):
    images, captions, lengths = zip(*batch)
    
    # Find the maximum length of captions in the batch
    max_length = max(lengths)
    
    # Pad the captions to the maximum length
    captions_padded = torch.zeros(len(captions), max_length).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        captions_padded[i, :end] = cap[:end]
    
    # Stack the tensors
    images = torch.stack(images, dim=0)
    
    return images, captions_padded, lengths

In [16]:
train_data, valid_data = random_split(dataset, (train_size, val_size))

train_loader = DataLoader(train_data, batch_size=Config.batch_size, shuffle=True, collate_fn=pad_collate)
valid_loader = DataLoader(valid_data, batch_size=Config.batch_size, shuffle=False, collate_fn=pad_collate)

print(len(train_data))

80


In [17]:
# Models
from einops.layers.torch import Rearrange

class ConvNetEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Dropout2d(),
#             Rearrange("b h w c -> b c h w"),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 256),
            nn.Linear(256, 256),
            nn.BatchNorm1d(128, momentum=0.01),
        )

    def forward(self, image):
        encoded_image = self.conv_net(image)

        return encoded_image


class PretrainedConvNet(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        resnet = models.resnet152(pretrained=True)
        resnet_modules = list(resnet.children())[:-1]

        self.resnet = nn.Sequential(*resnet_modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.batch_norm = nn.BatchNorm1d(embed_size, momentum=0.01)

    def forward(self, images):
        with torch.no_grad():
            features = self.resnet(images)

        features = features.reshape(features.size(0), -1)
        features = self.batch_norm(self.linear(features))

        return features


class TextRNNDecoder(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=5, max_len=30):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_len = max_len

    def forward(self, features, captions, lengths):
        text_embed = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), text_embed), dim=1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted=False)
        hidden, _ = self.lstm(packed)
        rnn_output = self.linear(hidden[0])

        return rnn_output

    def sample(self, features, state=None):
        sampled = []
        input_seq = features.unsqueeze(1)
        for _ in range(self.max_len):
            hidden, state = self.lstm(input_seq, state)
            outputs = self.linear(hidden.squeeze(1))
            _, predicted = outputs.max(1)
            sampled.append(predicted)
            input_seq = self.embed(predicted)
            input_seq = input_seq.unsqueeze(1)

        sampled = torch.stack(sampled, 1)

        return sampled

In [18]:
class ImageTextModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.vision_encoder = PretrainedConvNet(embed_size=Config.embed_size)
        self.text_decoder = TextRNNDecoder(
            vocab_size=vocab_size,
            embed_dim=Config.embed_size,
            hidden_size=Config.hidden_size,
        )

    def forward(self, image, text, lengths):
        visual_tokens = self.vision_encoder(image)
        decoder_output = self.text_decoder(visual_tokens, text, lengths)

        return decoder_output


In [19]:
wandb_config = {
    'num_epochs': 30,
    'lr': 0.01
}



In [20]:
def display_image(image):
    plt.figure(figsize=(50, 50))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [21]:
import wandb

wandb.login(key='bb29015546635cd4cab7430dbd6e815ad305f481')

wandb.init(project="shiryoku_vision", name='test_run_1ka', config=wandb_config)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

shiryoku_model = ImageTextModel()
shiryoku_model = shiryoku_model.to(device)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tensorkelechi (tensor_airl). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240525_164339-6c4qc409
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run test_run_1ka
wandb: ⭐️ View project at https://wandb.ai/tensor_airl/shiryoku_vision
wandb: 🚀 View run at https://wandb.ai/tensor_airl/shiryoku_vision/runs/6c4qc409
/opt/conda/lib/python3.10/site-packages/torchvision/model

In [22]:
wandb.watch(shiryoku_model, log_freq=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=shiryoku_model.parameters(), lr=Config.lr)
epochs = Config.num_epochs


In [23]:
os.mkdir(Config.model_output_path)

In [24]:
image_folder = '/kaggle/input/test-shiryoku/test_images'

In [25]:
def sample_run(image_file, model, device):
    image = pillow_image.open(image_file)
    image = read_img(image)
    model.eval()
    
    model = model.to(device)
    image = torch.tensor(image, dtype=torch.float32).to(device)
    image = image.unsqueeze(0) 
    image = rearrange(image, "b h w c -> b c h w")
    
    image_tensor = image.to(device)
    
    model_prediction = model.vision_encoder(image_tensor)

    model_prediction = model_prediction.squeeze(1)

    model_prediction = model_prediction.to(device)

    sampled_ids = model.text_decoder.sample(model_prediction)
    
    sampled_ids = sampled_ids[0].cpu().numpy()
    
    sampled_caption = []
    
    for word_id in sampled_ids:
        word = idx2word[word_id]
        sampled_caption.append(word)
        if word == "<end>":
            break
            
    caption = " ".join(sampled_caption)

    print(f'Image file: {image_file}')
    print(caption)
    
#     display_image(image_file)
#     wandb.log({"example": wandb.Image(image_file), "caption": caption})


In [26]:

def train_step(train_loader, model):
    total_correct = 0
    total_samples = 0
    model.train()
    for _, (images, captions, lengths) in tqdm(enumerate(train_loader)):
        
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True, enforce_sorted=False)[0]
        
        model_outputs = model(images, captions, lengths)
        
        _, predicted = torch.max(model_outputs, 1)

        # Update the running total of correct predictions and samples
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)
        
        accuracy = 100 * total_correct / total_samples
        train_loss = criterion(model_outputs, targets)
        
        model.zero_grad()
        train_loss.backward()
        optimizer.step()
    
    return accuracy, train_loss

def validation_step(model, valid_loader):
    val_loss = 0.0
    total_correct = 0
    total_samples = 0
    model.eval() 
    
    with torch.no_grad():  
        for _, (images, captions, lengths) in tqdm(enumerate(valid_loader)):
            images = images.to(device)
            captions = captions.to(device)
            targets = pack_padded_sequence(captions, lengths, batch_first=True, enforce_sorted=False)[0]

        # Update the running total of correct predictions and samples
            model_outputs = model(images, captions, lengths)
            
            _, predicted = torch.max(model_outputs, 1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)
            
            val_accuracy = 100 * total_correct / total_samples
            
            loss = criterion(model_outputs, targets)
            val_loss += loss.item()
    
    return val_accuracy, val_loss



def training_loop(model, train_loader, valid_loader, epochs=epochs):
    model.train()
    for epoch in tqdm(range(epochs)):
        print(f'Training epoch {epoch}')
        train_acc, train_loss = train_step(train_loader, model)
        valid_acc, valid_loss = validation_step(model, valid_loader)
        
        for image_file in os.listdir(image_folder):
            sample_run(os.path.join(image_folder, image_file), model, device)
        
        print(
            f"Epoch {epoch} of {epochs}, train_accuracy: {train_acc:.2f}, train_loss: {train_loss.item():.4f}, valid_accuracy: {valid_acc:.2f}, val_loss: {train_loss.item():.2f}"
        )
        
        if epoch % 5 == 0:
            torch.save(model.state_dict(), os.path.join(os.getcwd(), Config.model_output_path, f'caption_model_{epoch}.pth'))
            print(f'Saved model checkpoint @ epoch {epoch}')

        wandb.log({"accuracy": train_acc, "loss": train_loss, "valid_accuracy": valid_acc, "val_loss": valid_loss})
        print(f"Epoch {epoch} complete!")

    print(
        f"End metrics for run of {epochs}, accuracy: {train_acc:.2f}, train_loss: {train_loss.item():.4f},valid_accuracy: {valid_acc:.2f}, valid_loss: {valid_loss:.4f}"
    )
    
    torch.save(model.state_dict(), os.path.join(os.getcwd(), Config.model_output_path, f'{Config.model_filename}'))


In [27]:
training_loop(shiryoku_model,train_loader, valid_loader)

  0%|          | 0/35 [00:00<?, ?it/s]

Training epoch 0


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Image file: /kaggle/input/test-shiryoku/test_images/hamilton_f1.jpeg
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/woods.png
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/katara.jpeg
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/fireship_space.jpeg
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/windows_trash.jpeg
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Epoch 0 of 35, train_accuracy: 29.94, train_loss: 5.4234, valid_accuracy: 10.72, val_loss: 5.42
Saved model checkpoint @ epoch 0
Epoch 0 complete!
Training epoch 1


...

0it [00:00, ?it/s]

Image file: /kaggle/input/test-shiryoku/test_images/hamilton_f1.jpeg
<pad> _ <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Image file: /kaggle/input/test-shiryoku/test_images/woods.png
[s] _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/katara.jpeg
[s] _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/fireship_space.jpeg
[s] _ _ _ _ conference _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/windows_trash.jpeg
[s] _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Epoch 33 of 35, train_accuracy: 79.51, train_loss: 0.9974, valid_accuracy: 81.26, val_loss: 1.00
Epoch 33 complete!
Training epoch 34


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Image file: /kaggle/input/test-shiryoku/test_images/hamilton_f1.jpeg
[s] _ _ _ <pad> <pad> _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/woods.png
[s] _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/katara.jpeg
[s] _ _ _ _ art _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/fireship_space.jpeg
[s] _ _ _ _ conference _ flustered _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Image file: /kaggle/input/test-shiryoku/test_images/windows_trash.jpeg
[s] _ _ _ _ _ man _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Epoch 34 of 35, train_accuracy: 79.67, train_loss: 0.9747, valid_accuracy: 81.32, val_loss: 0.97
Epoch 34 complete!
End metrics for run of 35, accuracy: 79.67, train_loss: 0.9747,valid_accuracy: 81.32, valid_loss: 77.5625
